In [1]:
import stormpy
import stormpy.core
import stormpy.pars
import stormpy.examples
import numpy as np
from scipy.optimize import linprog

def create_model(prism_program_path, parameter_values):
    """
    Helper function to create the model based on the provided input
    """
    orig_program = stormpy.parse_prism_program(prism_program_path)
    program = orig_program.define_constants(
        stormpy.parse_constants_string(
            orig_program.expression_manager, parameter_values
        )
    )
    options = stormpy.BuilderOptions(True, True)
    options.set_build_state_valuations()
    options.set_build_choice_labels()
    model = stormpy.build_sparse_model_with_options(program, options)
    return model

In [2]:
def hoffman_karp(prism_program_path, parameter_values):
    """
    Shapley's algorithm. Due to the simplifications to the problem, using this algorithm is essentially the
    same as running value iteration until convergence once for the entire model

    Parameters:
        prism_program_path (str): Path to the PRISM program file
        parameter_values: values for variables in model
        gamma (float): Discount factor
        epsilon (float): Tolerance for convergence

    Returns:
        np.ndarray: Value function for all states
    """

    # Parsing of the model
    model = create_model(prism_program_path, parameter_values)

    # Define the rewards for each state
    properties = stormpy.parse_properties('Rmax=? [F"goal"]')
    result = stormpy.model_checking(model, properties[0], extract_scheduler=True)
    return result

In [14]:
def shapley(
    prism_program_path, parameter_values, gamma=0.85, epsilon=1e-6
):
    """
    Shapley's algorithm 

    Parameters:
        prism_program_path (str): Path to the PRISM program file
        parameter_values: values for variables in model
        gamma (float): Discount factor
        epsilon (float): Tolerance for convergence

    Returns:
        np.ndarray: Value function for all states
    """
    # Parse the PRISM program
    model = create_model(prism_program_path, parameter_values)

    # Extract states and actions
    num_states = model.nr_states
    num_actions1 = max(len(s.actions) for s in model.states)
    num_actions2 = max(len(s.actions) for s in model.states)

    # Extract transitions and rewards
    reward_model_name = list(model.reward_models.keys())[0]
    state_action_rewards = model.reward_models[reward_model_name].state_action_rewards
    model_rewards = {}
    ind = 0
    for state in model.states:
        for action in state.actions:
            model_rewards[(state.id, action.id)] = state_action_rewards[ind]
            ind += 1

    transitions = np.zeros((num_states, num_actions1, num_actions2, num_states))
    rewards = np.zeros((num_states, num_actions1, num_actions2))

    for s, state in enumerate(model.states):
        choices = state.actions
        for i, choice1 in enumerate(choices):
            for j, _ in enumerate(choices):
                for transition in choice1.transitions:
                    target_state = transition.column
                    probability = transition.value()
                    transitions[s, i, j, target_state] += probability
                    rewards[s, i, j] += model_rewards[
                        (state.id, choice1.id)
                    ]  # Assuming single reward structure

    # Initialize the value function
    V = np.zeros(num_states)

    while True:
        V_prev = V.copy()
        Q = np.zeros((num_states, num_actions1, num_actions2))

        # Compute Q-values for all state-action pairs
        for s in range(num_states):
            for a1 in range(num_actions1):
                for a2 in range(num_actions2):
                    Q[s, a1, a2] = rewards[s, a1, a2] + gamma * np.dot(
                        transitions[s, a1, a2], V_prev
                    )

        # Solve the robust optimization for each state
        for s in range(num_states):
            c = -Q[
                s
            ].flatten()  # Objective function coefficients (minimize -Q is the same as maximize Q)
            A_eq = np.ones((1, num_actions1 * num_actions2))
            b_eq = [1]

            bounds = [(0, 1) for _ in range(num_actions1 * num_actions2)]

            res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

            V[s] = -res.fun  # Value of the game for state s

        # Check for convergence
        if np.max(np.abs(V - V_prev)) < epsilon:
            break

    return V

In [63]:
param_vals = ["p=0.1,q=0.4,m=0.6,n=0.2,l=0.1",
            "p=0.7,q=0.6,m=0.8,n=0.5,l=0.9",
            "p=0.4,q=0.3,m=0.5,n=0.6,l=0.7",
            "p=0.5,q=0.4,m=0.6,n=0.7,l=0.8",
            "p=0.2,q=0.5,m=0.6,n=0.4,l=0.7",
            "p=0.3,q=0.4,m=0.5,n=0.6,l=0.7",
            "p=0.5,q=0.3,m=0.4,n=0.6,l=0.2",
            "p=0.6,q=0.3,m=0.4,n=0.8,l=0.5",
            "p=0.4,q=0.6,m=0.3,n=0.7,l=0.5",
            "p=0.5,q=0.4,m=0.6,n=0.3,l=0.7"]

for i in range(0,10):
    print(f"Running test {i+1}")
    prism_model_path = f"test/test{i+1}.prism"
    params = param_vals[i]
    print(f"HK: {hoffman_karp(prism_model_path,params)}")
    print(f"SH: {shapley(prism_model_path,params)}")

Running test 1
HK: {5.17391, 3, 4.95652, 0}
SH: [ 5.50012392  3.53267611  4.50787169 -0.        ]
Running test 2
HK: {inf, inf, inf, 0}
SH: [77.18388455 63.6063011  75.83582813 -0.        ]
Running test 3
HK: {11.6667, 16.6667, 0, 13.6667}
SH: [17.12517561 20.95639893 -0.         21.12517561]
Running test 4
HK: {inf, inf, inf, 0}
SH: [22.91288462 14.47595126 20.6131905  -0.        ]
Running test 5
HK: {8.01887, 10.0943, 0, 8.20755}
SH: [14.83581879 16.68128786 -0.         15.04417825]
Running test 6
HK: {inf, inf, inf, inf, 0}
SH: [24.59686962 14.9073384  11.42100559  6.70785475 -0.        ]
Running test 7
HK: {20, 15, 15, 13, 0}
SH: [24.21523167 14.20371182 19.24389291 14.35730849 -0.        ]
Running test 8
HK: {inf, inf, inf, 0}
SH: [48.41015414 45.3680929  44.91890424 -0.        ]
Running test 9
HK: {inf, inf, inf, 0}
SH: [30.00283085 24.68386266 26.68689819 -0.        ]
Running test 10
HK: {inf, inf, inf, 0}
SH: [20.99439159 19.92735288 15.35356965 -0.        ]
